# BERT for Cloze problem in python 2
Since the source codes provided for ScratchGan are in python 2 and are used with Tensorflow 1.x, let's try follow that style

## 1. Loading and Initializing

### 1.1 Sentence processing functions

In [4]:
import tensorflow as tf
import tensorflow_hub as hub
print("Using Tensorflow version: " + tf.__version__)

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices()[-1])

BERT_DIR = "/home/aufish/Downloads/bert"

Using Tensorflow version: 1.14.0
name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 6856667956
locality {
  bus_id: 1
  links {
  }
}
incarnation: 11857415821721002533
physical_device_desc: "device: 0, name: GeForce RTX 2070, pci bus id: 0000:01:00.0, compute capability: 7.5"



In [5]:
bert_module = hub.KerasLayer(BERT_DIR, trainable=True)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [7]:
# tokenizer
from bert import tokenization

def create_tokenizer(vocab_file, do_lower_case=False):
    return tokenization.FullTokenizer(vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer(BERT_DIR + "/assets/vocab.txt")

def convert_sentence_to_features(sentence, tokenizer, max_seq_len=50):
    tokens = ['[CLS]']
    tokens.extend(tokenizer.tokenize(sentence))
    if len(tokens) > max_seq_len-1:
        tokens = tokens[:max_seq_len-1]
    tokens.append('[SEP]')+63
    
    segment_ids = [0] * len(tokens)
    input_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_mask = [1] * len(input_ids)

    #Zero Mask till seq_length
    zero_mask = [0] * (max_seq_len-len(tokens))
    input_ids.extend(zero_mask)
    input_mask.extend(zero_mask)
    segment_ids.extend(zero_mask)
    
    return input_ids, input_mask, segment_ids

def convert_sentences_to_features(sentences, tokenizer, max_seq_len=50):
    all_input_ids = []
    all_input_mask = []
    all_segment_ids = []
    
    for sentence in sentences:
        input_ids, input_mask, segment_ids = convert_sentence_to_features(sentence, tokenizer, max_seq_len)
        all_input_ids.append(input_ids)
        all_input_mask.append(input_mask)
        all_segment_ids.append(segment_ids)
    
    return all_input_ids, all_input_mask, all_segment_ids

import random, copy
import numpy as np
def make_rand_mask(input_ids, input_mask, vocab_size, segment_id_vals=None):
    ''' 
    Only make mask for one sentence
    input_ids: the ids of words in the sentences
    input_mask: initial mask (1 if there is a word; 0 for padding)
    returns
    input_mask: replace one bit of 1 with 0, meaning that the word will be masked
    mask_word_ids: the id of words that are masked
    pure_ids: ids in number instead of one-hot (to generate weights per masked word)
    segment_id_vals: mark the masked word with segment id 1
    sequential_output_filter: the masked word index
    '''
    
    new_input_mask = copy.deepcopy(input_mask)
#     mask_word_ids = np.zeros(vocab_size)
    pure_ids = []

    total_word = sum(input_mask)
    mask_word = random.randint(0, total_word-1)

    pure_ids.append(input_ids[mask_word])
    assert new_input_mask[mask_word] == 1
    new_input_mask[mask_word] = 0
#     mask_word_ids[input_ids[mask_word]] = 1.0
                
    return new_input_mask, segment_id_vals, mask_word

### 1.2 Blank filler model

In [8]:
class WordPredictor(tf.keras.Model):
    # The output means, how possible the given word may fit into the blank
    def __init__(self, class_num, bert=bert_module, dropout=0.1):
        super(WordPredictor, self).__init__()
        self.bert = bert
        self.drop = tf.keras.layers.Dropout(rate=dropout, trainable=True)
        
        self.dense = tf.keras.layers.Dense(
            class_num,
            activation=None,
            kernel_initializer='glorot_uniform',
            name='word_prediction',
            trainable=True)
        
    def call(self, inputs, mask_loc):
        # When passed in, all tensors are stacked in one, split it into a list
        inputs = tf.unstack(tf.cast(inputs, tf.dtypes.int32), axis=1)
        
#         # The last element in the list is a filter that will be exerted on sequential
#         inputs, seq_output_filter = inputs[:-1], inputs[-1]
#         seq_output_filter = tf.cast(seq_output_filter, tf.dtypes.float32)

        pooled, sequential = self.bert(inputs)
        
#         s = tf.tensordot(seq_output_filter,  sequential, axes=(1, 1))

        # select one from each batch
        s = tf.gather_nd(sequential, [(i, mask_loc[i]) for i in range(sequential.shape[0])])
        # s now has shape (batch_size * 768)
        
        x = self.drop(s)
        return self.dense(x)

### 1.2.1 Sanity test

In [11]:
model = WordPredictor(1)

opt = tf.keras.optimizers.Adam(learning_rate=1e-4)
model.compile(opt)

## 2. Prepare data

In [12]:
import numpy as np
# extracted from emnlp
DATA_FILE = "./sentences.txt"

MASK_ID = tokenizer.convert_tokens_to_ids(['[MASK]'])[0]

def parse_line(line):    
    input_ids, input_mask, segment_ids = convert_sentence_to_features(line, tokenizer, max_seq_len=30)
        
    input_mask, segment_ids, mask_position = \
        make_rand_mask(input_ids, input_mask, len(tokenizer.vocab), segment_ids)
    
    label = input_ids[mask_position]
    input_ids[mask_position] = MASK_ID
    
    return [input_ids, input_mask, segment_ids], [mask_position, label]

def create_dataset(filename = DATA_FILE, data_size = 268528, batch_size = 10):
    dataset = tf.data.TextLineDataset([filename])
    
    dataset = dataset.map(lambda x : tf.numpy_function(parse_line, [x], [tf.int64, tf.int64]))
    
    # dataset = dataset.shuffle(data_size, reshuffle_each_iteration=True)
    
    dataset = dataset.batch(batch_size)
    
    return dataset

In [13]:
tokenizer.convert_tokens_to_ids(['[MASK]'])

[103]

### 2.1 Model calling sanity test

In [14]:
temp_batch_size = 3
dataset = create_dataset(batch_size = temp_batch_size)

In [15]:
# Processing data without model
for (bert_input, label) in dataset.take(1):
    print(bert_input)
    print(label)
    inputs = tf.unstack(tf.cast(bert_input, tf.dtypes.int32), axis=1)
    
    pooled, sequential = bert_module(inputs)
    
    print(sequential.shape)
    
#     seq_output_filter = tf.cast(seq_output_filter, tf.dtypes.float32)
#     r = tf.tensordot(seq_output_filter,  sequential, axes=(1, 1))
    
    mask, label = tf.unstack(label, axis=1)
    r = tf.gather_nd(sequential, [(i, mask[i]) for i in range(temp_batch_size)])
    
    print(r)
    print(label)

Tensor("IteratorGetNext:0", dtype=int64)
Tensor("IteratorGetNext:1", dtype=int64)


ValueError: Cannot infer num from shape <unknown>